In [12]:
from sage.all import *
import hashlib


$$\left\{\begin{array}{rcl} s &\gets& \mathsf{Zr} \\ \beta & \gets& g(s)\end{array}\right.$$

In [13]:
G=None

def KeyGen(lmbda):
    global G
    q = next_prime(2^lmbda)
    F = GF(q)
    G = F.multiplicative_generator()
    
    s = randint(1, q-1)
    beta = G ^ s


    return (s, beta)

$$key\;\gets\;(\,\vartheta \;d \gets \mathsf{ID}(\mathit{id})\;\centerdot\; s* d\,)$$

In [14]:
def KeyExtract(id, s):
    d = ascii_integer(id)
    key = s * d
    return key



$$\mathsf{in\_ encrypt}(\mathit{id},x)\;\equiv$$ 
$$\vartheta\,d\gets\mathsf{ID}(\mathit{id})\,\centerdot\,\vartheta\,v\gets\mathsf{Zr}\,\centerdot\,\vartheta\,a\gets H(v\oplus x)\,\centerdot\,\vartheta\,\mu\gets\mathbf{ex}(\beta,d,a)\,\centerdot\,\langle x,v,a,\mu\rangle$$

In [15]:
def in_encrypt(id, x, beta):
    d = ascii_integer(id)
    v = random_zmod_element(beta.order())
    a = hash(v + ascii_integer(x))
    mu = beta^d * G^a
    return (x, v, a, mu)

$$\mathsf{out\_ encrypt}(x,v,a,\mu)\;\equiv$$
$$\vartheta\,\alpha\gets g(a)\,\centerdot\,\vartheta\,v'\gets v\oplus f(\mu)\,\centerdot\,x'\gets x\oplus H(v)\,\centerdot\,\langle \alpha,v',x'\rangle$$

In [16]:
def out_encrypt(x, v, a, mu):
    alpha = G^a
    v_prime = v ^ mu
    x_prime = x ^ hash(v)
    return (alpha, v_prime, x_prime)



$$\mathsf{Encrypt}(\mathit{id}, x) \;\equiv\;$$  
$$\vartheta \,x,v,a,\mu \gets \mathsf{in}(\mathit{id},x)\,\centerdot\,\mathsf{out}(x,v,a,\mu)$$

In [17]:
def Encrypt(id, x, beta):
    x, v, a, mu = in_encrypt(id, x, beta)
    alpha, v_prime, x_prime = out_encrypt(x, v, a, mu)
    return (alpha, v_prime, x_prime)


$$\mathsf{in\_ decrypt}(\mathit{key},\alpha, v', x')\;\equiv$$
$$\vartheta\,\mu \gets \mathbf{ex}(\alpha,\mathit{key},1)\,\centerdot\,\vartheta\,v \gets v'\oplus f(\mu)\,\centerdot\,x\gets x'\oplus H(v)\,\centerdot\,\langle \alpha,v,x\rangle$$

In [18]:
def in_decrypt(key, alpha, v_prime, x_prime):
    mu = alpha^key
    v = v_prime ^ mu
    x = x_prime ^ hash(v)
    return (alpha, v, x)



$$\mathsf{out\_ decrypt}(\alpha,v,x)\;\equiv$$
$$\vartheta\,a\gets H(v\oplus x)\,\centerdot\,\mathsf{if}\;\;\alpha\neq g(a)\;\;\mathsf{then}\;\;\texttt{fails}\;\;\mathsf{else}\;\;x$$

In [19]:
def out_decrypt(alpha, v, x):
    a = hash(v + x)
    if G^a != alpha:
        return "Decryption failed"
    return x

$$\mathsf{Decrypt}(\mathit{key}, c) \;\equiv\;$$
$$\vartheta \,\alpha,v,x \gets \mathsf{in}(\mathit{key},c)\,\centerdot\,\mathsf{out}(\alpha,v,x)$$

In [20]:

def Decrypt(key, alpha, v_prime, x_prime):
    alpha, v, x = in_decrypt(key, alpha, v_prime, x_prime)
    return out_decrypt(alpha, v, x)

In [23]:
lmbda = 128
id = "sender@test.com"
x = b'ola'

s, beta = KeyGen(lmbda)
key = KeyExtract(id, s)
alpha, v_prime, x_prime = Encrypt(id, x, beta)
decrypted_x = Decrypt(key, alpha, v_prime, x_prime)
print("Original message:", x)
print("Decrypted message:", decrypted_x)

TypeError: Cannot convert int to sage.structure.parent.Parent

In [ ]:
# Geração dos primos  q, p
bq     = 192                # tamanho em bits do primo "q"
bp     = 768                # tamanho minimo em bits do primo "p"

q = random_prime(2^bq-1,lbound=2^(bq-1))

t = q*3*2^(bp - bq)
while not is_prime(t-1):
    t = t << 1

p = t - 1

# Aneis e Corpos
Fp     = GF(p)                  # corpo primo com "p" elementos
R.<z>  = Fp[]                   # anel dos polinomios em "z" de coeficientes em Fp
f     = R(z^2 + z + 1)
Fp2.<z> = GF(p^2, modulus=f)   
# extensão de Fp de dimensão 2 cujo módulo é o polinómio "f"
# o polinómio "f"  é irredutivel, tem grau 2 e verifica  z^3 = 1 mod f
# se o ponto (x,y) verificar a equação y^2 = x^3 + 1, 
#      então o ponto (z*x,y) verifica a mesma equação

def trace(x):       # função linear que mapeia Fp2  em  Fp
    return x + x^p

# Curvas Elipticas supersingulares em Sagemath

# a curva supersingular sobre Fp2  definida pela equação  y^2 = x^3 + 1
E2 = EllipticCurve(Fp2, [0,1])

# ponto arbitrário  de ordem "q" em E2        
cofac = (p + 1)//q
G = cofac * E2.random_point()

# emparelhamento e oraculo DDHP

def phi(P):             # a isogenia que mapeia  (x,y)  ->  (z*x,y)
    (x,y) = P.xy()
    return E2(z*x,y)

def TateX(P,Q,l=1):      # o emparelhamento de Tate generalizado
    return P.tate_pairing(phi(Q), q, 2)^l

def ddhp(P,Q,R):        # o oraculo DDHP  que decide se (P,Q,R) é um triplo de DH
    return tateX(P,Q) == tateX(R,G)